# Final exam code

The source code for resolving the exercises of the final exam for GRA4153 question 3

For this code, the following libraries are necessary:
- Pandas

The funcitons necessary for this notebook are found in the following modules

In [11]:
%load_ext autoreload
%autoreload 2
import DataImporter as get

## Q3.i
Selecting the correct sample for evaluating

In [12]:
indpro = get.Data("indpro.csv")
dir(indpro)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_raw_data',
 '_sample',
 'sample']